In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

# Load measurement data
measurements_df = pd.read_excel("measurements.xlsx")

# Function to detect primary root landmarks
def detect_primary_root_landmarks(image_path, primary_root_tip_gt, junction_gt):
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply thresholding to segment roots
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Find centroid of largest contour (primary root tip)
    primary_root_tip = None
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            primary_root_tip = (cx, cy)
    
    # Find junction between root and hypocotyl
    junction = None
    if primary_root_tip:
        # Hypothetically, we might use some algorithm to find the junction based on primary root tip location
        # For simplicity, we'll assume it's 50 pixels above the primary root tip
        junction = (primary_root_tip[0], primary_root_tip[1] - 50)
    
    # Calculate errors
    primary_root_tip_error = np.sqrt((primary_root_tip[0] - primary_root_tip_gt[0])**2 + (primary_root_tip[1] - primary_root_tip_gt[1])**2)
    junction_error = np.sqrt((junction[0] - junction_gt[0])**2 + (junction[1] - junction_gt[1])**2)
    
    return primary_root_tip, junction, primary_root_tip_error, junction_error

# Paths to images
image_paths = ["measurement_image_1.tif", "measurement_image_2.tif", "measurement_image_3.tif"]

# Ground truth landmark coordinates (from measurements.xlsx)
primary_root_tip_gt = [(int(measurements_df['V1'][0]), int(measurements_df['V2'][0])),
                       (int(measurements_df['V1'][1]), int(measurements_df['V2'][1])),
                       None]  # No ground truth provided for image 3

junction_gt = [(int(measurements_df['V1'][2]), int(measurements_df['V2'][2])),
               (int(measurements_df['V1'][3]), int(measurements_df['V2'][3])),
               None]  # No ground truth provided for image 3

# Loop over images
for i, image_path in enumerate(image_paths):
    # Detect primary root landmarks
    primary_root_tip, junction, primary_root_tip_error, junction_error = detect_primary_root_landmarks(image_path, primary_root_tip_gt[i], junction_gt[i])
    
    # Print errors
    print(f"Image {i+1}:")
    print(f"Primary Root Tip Error: {primary_root_tip_error:.2f} pixels")
    print(f"Junction Error: {junction_error:.2f} pixels")
    
    # Visualize
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE), cmap='gray')
    plt.scatter(primary_root_tip[0], primary_root_tip[1], c='r', s=50, label='Primary Root Tip')
    plt.scatter(junction[0], junction[1], c='g', s=50, label='Junction')
    plt.title(f"Image {i+1}")
    plt.legend()
    plt.show()
